In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("lavin_results/meanshift_all_bin_w_seeding.csv")

In [5]:
for pair in zip(list(df["docid"]), list(df["group_label"])):
    
    

,Unnamed: 0,docid,group_label
0,0,1,1
1,1,2,1
2,2,3,1
3,3,4,3
4,4,5,1
5,5,6,1
6,6,7,1
7,7,8,1
8,8,9,1
9,9,10,1
